In [ ]:
import pandas as pd
from azure.storage.blob import BlobServiceClient
from io import StringIO

# Azure connection details
connection_string = "DefaultEndpointsProtocol=https;AccountName=lhindstorage;AccountKey=SPFV+Hdv7AWTGTpysBBwEVXJvefoYSSr17wLSWB+onc3PMYbwXEcpFRZvHvXF06eePtCy00PIvZu+AStpz1VzA==;EndpointSuffix=core.windows.net"
container_name = "clean-data"

In [ ]:
# File names in the container
files = {
    "order_items": "order_items.csv",
    "orders": "orders.csv",
    "order_payments": "order_payments.csv"
}

# Initialize BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

# Function to load a CSV file fron Azure Blob Storage into a Dataframe
def load_csv_from_blob(blob_name):
    blob_client = container_client.get_blob_client(blob_name)
    download_stream = blob_client.download_blob()
    content = download_stream.readall().decode("utf-8")
    return pd.read_csv(StringIO(content))

# Load datasets
order_items = load_csv_from_blob(files["order_items"])
orders = load_csv_from_blob(files["orders"])
order_payments = load_csv_from_blob(files["order_payments"])

In [ ]:
## Transformime
# Total_price
order_items['total_price'] = order_items['price'] + order_items['freight_value']

# Delivery time
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])

delivery_times = orders[['order_id', 'order_purchase_timestamp', 'order_delivered_customer_date']].copy()
delivery_times['delivery_time_days'] = (
    delivery_times['order_delivered_customer_date'] - delivery_times['order_purchase_timestamp']
).dt.days

order_items = order_items.merge(delivery_times[['order_id', 'delivery_time_days']], on='order_id', how='left')

# Payment count
order_payments_sum = order_payments.groupby('order_id')['payment_installments'].sum().reset_index()
order_payments_sum.rename(columns={'payment_installments': 'payment_count'}, inplace=True)
order_payments_sum['payment_count'] = order_payments_sum['payment_count'].astype(int)

order_items = order_items.merge(order_payments_sum, on='order_id', how='left')

# Profit margin
order_items['profit_margin'] = order_items['price'] - order_items['freight_value']

In [ ]:
## Upload 
# Convert DataFrame to CSV
csv_buffer = StringIO()
order_items.to_csv(csv_buffer, index=False)
csv_data = csv_buffer.getvalue()

# Upload the updated CSV
output_blob_name = "order_items.csv"
blob_client = container_client.get_blob_client(output_blob_name)
blob_client.upload_blob(csv_data, overwrite=True)